In [ ]:
# python 3.10.9에서 작성 되었습니다.

konlpy 사용시 참고 링크에 들어가서 설치하세요

참고 : https://wikidocs.net/22488

In [ ]:
# pip install kss

In [ ]:
# pip install konlpy

In [ ]:
# pip install tensorflow

In [ ]:
# 패키지 불러오기
import kss
import nltk
import pandas as pd
import numpy as np
import tensorflow
import re
from statannot import add_stat_annotation
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.tokenize import WordPunctTokenizer
from nltk.tokenize import TreebankWordTokenizer
from konlpy.tag import Hannanum
from konlpy.tag import Okt
from konlpy.tag import Kkma
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

hannanum = Hannanum()
kkma = Kkma()
okt = Okt()
tfidf = TfidfVectorizer()
tokenizer = Tokenizer()

In [ ]:
print('OKT 형태소 분석 :',okt.morphs("열심히 코딩한 당신, 연휴에는 여행을 가봐요"))
print('OKT 품사 태깅 :',okt.pos("열심히 코딩한 당신, 연휴에는 여행을 가봐요"))
print('OKT 명사 추출 :',okt.nouns("열심히 코딩한 당신, 연휴에는 여행을 가봐요")) 

In [ ]:
# 로우 데이터 불러오기 = df
df = pd.read_excel('dataframe.xlsx', index_col = 0)
df.head(2)

In [ ]:
# 컬럼명 변경
df.columns = ['제목', '연수명', '분류', '만족도', '내용']
df.head(2)

In [ ]:
# 결측치 확인 True가 있으면 nan값 있음
df['내용'].isna().unique()

In [ ]:
tfidf.fit(df['내용'])
tfidf.vocabulary_

In [ ]:
# label : 1 = 만족도 5, 4 
# label : 0 = 만족도 3, 2, 1
df['label'] = df['만족도'] >= 4
df['label'] = df['label'].astype(int)

In [ ]:
# 만족도 5, 4 = df1
df1 = df[df['만족도'].isin([5, 4])]
df1

In [ ]:
# 만족도 3, 2, 1 = df2
df2 = df[df['만족도'].isin([3, 2, 1])]
df2

In [ ]:
#전처리 함수 만들기
def preprocessing(review, okt, remove_stopwords = False, stop_words =[]):
    #함수인자설명
    # review: 전처리할 텍스트
    # okt: okt객체를 반복적으로 생성하지 않고 미리 생성 후 인자로 받음\
    # remove_stopword: 불용어를 제거할지 여부 선택. 기본값 False
    # stop_words: 불용어 사전은 사용자가 직접 입력, 기본값 빈 리스트

    # 1. 한글 및 공백 제외한 문자 모두 제거
    review_text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]','',review)

    #2. okt 객체를 활용하여 형태소 단어로 나눔
    word_review = okt.morphs(review_text,stem=True)

    if remove_stopwords:
        #3. 불용어 제거
        word_review = [token for token in word_review if not token in stop_words]
    return word_review

In [ ]:
# 불용어 제거 = https://www.ranks.nl/stopwords/korean
f = open("Korean Stopwords.txt", "r", encoding='UTF8')

lst = []
while True:
    line = f.readline().strip()
    if not line: break
    lst.append(line)

In [ ]:
# 줄바꿈 제거 추가
lst2 = ['\n', '\n\n', '\n\n\n', '\n\n\n\n', '\n\n\n\n\n']
lst.extend(lst2)

In [ ]:
# 로우 데이터 = df 전처리
stop_words = lst
clean_review_df = []

for review in df['내용']:
    # 리뷰가 문자열인 경우만 전처리 진행
    if type(review) == str:
        clean_review_df.append(preprocessing(review,okt, remove_stopwords = True, stop_words = stop_words))
    else:
        clean_review_df.append([]) #str이 아닌 행은 빈칸으로 놔두기
clean_review_df

In [ ]:
# 만족도 5, 4 = df1 전처리
stop_words = lst
clean_review_df1 = []

for review in df1['내용']:
    # 리뷰가 문자열인 경우만 전처리 진행
    if type(review) == str:
        clean_review_df1.append(preprocessing(review,okt, remove_stopwords = True, stop_words = stop_words))
    else:
        clean_review_df1.append([]) #str이 아닌 행은 빈칸으로 놔두기
clean_review_df1

In [ ]:
# 만족도 3, 2, 1 = df2 전처리
stop_words = lst
clean_review_df2 = []

for review in df2['내용']:
    # 리뷰가 문자열인 경우만 전처리 진행
    if type(review) == str:
        clean_review_df2.append(preprocessing(review,okt, remove_stopwords = True, stop_words = stop_words))
    else:
        clean_review_df2.append([]) #str이 아닌 행은 빈칸으로 놔두기
clean_review_df2

In [ ]:
# 로우 데이터 불러오기 = df 
sequences = tokenizer.texts_to_sequences(clean_review_df)

word_vocab = tokenizer.word_index #단어사전형태
MAX_SEQUENCE_LENGTH = 8 #문장 최대 길이

#데이터
df_inputs = pad_sequences(sequences,maxlen=MAX_SEQUENCE_LENGTH,padding='post')

#데이터 라벨 벡터화
df_labels = np.array(df['label'])

In [ ]:
df_inputs

In [ ]:
df_labels